In [2]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm

In [3]:
years = ['2021','2022']

In [4]:
year = '2022'

max_code = 1093

In [5]:
question_info = pd.read_csv('MicrodadosAlunos/2022/ITENS_PROVA_2022.csv',sep=';',encoding='latin-1')

In [6]:
question_info = question_info[question_info['CO_PROVA']<=max_code]

In [7]:
df_by_question = question_info.groupby('CO_ITEM').agg({'CO_POSICAO':list,
                          'SG_AREA':'max',
                          'TX_GABARITO':'max',
                           'CO_HABILIDADE':'max',
                          'IN_ITEM_ABAN':'max',
                          'TX_MOTIVO_ABAN':'max',
                          'NU_PARAM_A':'max',
                          'NU_PARAM_B':'max',
                          'NU_PARAM_C':'max',
                          'TX_COR':list,
                          'CO_PROVA':list,
                           'TP_LINGUA':'max',
                          'IN_ITEM_ADAPTADO':list})
# df_by_question['RESPOSTAS_ESTUDANTES'] = [[]]*len(df_by_question)
# df_by_question['TP_ESCOLA_ESTUDANTES'] = [[]]*len(df_by_question)
# df_by_question['RENDA_ESTUDANTES'] = [[]]*len(df_by_question)
# df_by_question['UF_ESTUDANTES'] = [[]]*len(df_by_question)
# df_by_question['SEXO_ESTUDANTES'] = [[]]*len(df_by_question)
# df_by_question['RACA_ESTUDANTES'] = [[]]*len(df_by_question)
# df_by_question['NU_INSCRICAO'] = [[]]*len(df_by_question)
df_by_question.head()

,CO_POSICAO,SG_AREA,TX_GABARITO,CO_HABILIDADE,IN_ITEM_ABAN,TX_MOTIVO_ABAN,NU_PARAM_A,NU_PARAM_B,NU_PARAM_C,TX_COR,CO_PROVA,TP_LINGUA,IN_ITEM_ADAPTADO
CO_ITEM,,,,,,,,,,,,,
5961,"[143, 158, 161, 173, 173, 173, 173]",MT,D,7,0,NaN,0.90320,2.45135,0.16170,"[AZUL, AMARELA, ROSA, CINZA, CINZA, CINZA, VERDE]","[1075, 1076, 1077, 1078, 1079, 1080, 1083]",NaN,"[0, 0, 0, 0, 0, 0, 0]"
6864,"[40, 7, 12, 38, 38, 38, 38, 38, 38]",LC,D,16,0,NaN,0.57225,0.09790,0.01556,"[AZUL, AMARELA, ROSA, BRANCA, BRANCA, BRANCA, ...","[1065, 1066, 1067, 1068, 1069, 1070, 1071, 107...",NaN,"[0, 0, 0, 0, 0, 0, 0, 1, 0]"
8020,"[137, 137]",MT,E,12,0,NaN,2.19733,2.65319,0.11011,"[LARANJA, LARANJA]","[1081, 1082]",NaN,"[0, 1]"
8364,"[160, 178, 163, 154, 154, 154, 154]",MT,A,6,0,NaN,0.89111,2.43206,0.25086,"[AZUL, AMARELA, ROSA, CINZA, CINZA, CINZA, VERDE]","[1075, 1076, 1077, 1078, 1079, 1080, 1083]",NaN,"[0, 0, 0, 0, 0, 0, 0]"
9678,[21],LC,E,28,0,NaN,0.76200,-0.54000,0.21100,[VERDE],[1073],NaN,[0]


In [8]:
question_info.to_csv('MicrodadosAlunos/questao_prova_mapping.csv',index=False)

In [9]:
#create an answer mapping that maps code OF ENEM TEST and type of language to question codes
english_answers = question_info.copy()
english_answers = english_answers[english_answers['TP_LINGUA']!=1]
english_answers['TP_LINGUA']=0

spanish_answers = question_info.copy()
spanish_answers = spanish_answers[spanish_answers['TP_LINGUA']!=0]
spanish_answers['TP_LINGUA']=1

answer_mapping = pd.concat([english_answers,spanish_answers])
answer_mapping = answer_mapping[['CO_PROVA','TP_LINGUA','CO_POSICAO','CO_ITEM']]
answer_mapping = answer_mapping.sort_values(by=['CO_PROVA','TP_LINGUA','CO_POSICAO']).groupby(['CO_PROVA','TP_LINGUA']).agg(list)

In [10]:
answer_mapping

CO_POSICAO  \
CO_PROVA TP_LINGUA                                                      
1055     0          [46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 5...   
         1          [46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 5...   
1056     0          [46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 5...   
         1          [46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 5...   
1057     0          [46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 5...   
...                                                               ...   
1091     1          [91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101,...   
1092     0          [91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101,...   
         1          [91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101,...   
1093     0          [91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101,...   
         1          [91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101,...   

                                                              CO_ITEM  
CO_PROVA TP_LINGUA                                                     
1055     0          [140506, 83806, 141322, 140639, 140682, 87207,...  
         1          [140506, 83806, 141322, 140639, 140682, 87207,...  
1056     0          [141030, 140585, 112145, 84199, 141048, 140914...  
         1          [141030, 140585, 112145, 84199, 141048, 140914...  
1057     0          [96291, 140572, 96516, 140190, 111984, 140643,...  
...                                                               ...  
1091     1          [31313, 75845, 82765, 141503, 88655, 83901, 11...  
1092     0          [31313, 75845, 82765, 141503, 88655, 83901, 11...  
         1          [31313, 75845, 82765, 141503, 88655, 83901, 11...  
1093     0          [31313, 40571, 82765, 141503, 88655, 83901, 11...  
         1          [31313, 40571, 82765, 141503, 88655, 83901, 11...  

[72 rows x 2 columns]

In [11]:
#GET STUDENT DATA, IGNORE TRAINEES
student_info = pd.read_csv('MicrodadosAlunos/2022/MICRODADOS_ENEM_2022.csv',sep=';',encoding='latin-1')
student_info = student_info[(student_info['IN_TREINEIRO']==0) & (~student_info['CO_PROVA_CH'].isna()) & (~student_info['CO_PROVA_CN'].isna()) & (~student_info['CO_PROVA_LC'].isna())& (~student_info['CO_PROVA_MT'].isna())]
student_info = student_info[(student_info['CO_PROVA_CH']<=max_code) & (student_info['CO_PROVA_CN']<=max_code) & (student_info['CO_PROVA_LC']<=max_code)& (student_info['CO_PROVA_MT']<=max_code)].to_dict(orient='records')

In [12]:
#this maps question id to all info from people who answered it
student_answers = []
students=[]
student_hash=set()
student_id_mapping = {}

def fill_data_for_question(item,student,answer):
    if student['NU_INSCRICAO'] not in student_hash:
        student_hash.add(student['NU_INSCRICAO'])
        student_id_mapping[student['NU_INSCRICAO']] = len(student_hash)
        students.append({
            'ID':student_id_mapping[student['NU_INSCRICAO']],
            'NU_INSCRICAO':student['NU_INSCRICAO'],
            'SG_UF_PROVA':student['SG_UF_PROVA'],
            'TP_ESCOLA':student['TP_ESCOLA'],
            'RENDA':student['Q006'],
            'TP_SEXO':student['TP_SEXO'],
            'TP_COR_RACA':student['TP_COR_RACA'],
        })
    
    student_answers.append({
        'CO_ITEM':item,
        'ID':student_id_mapping[student['NU_INSCRICAO']],
        'RESPOSTA':answer,
    })
    

for std_id,student in enumerate(tqdm(student_info)):
    lingua = student['TP_LINGUA']
    if not student['TX_RESPOSTAS_CH']==np.nan:
        items_prova = answer_mapping.loc[(student['CO_PROVA_CH'],lingua),'CO_ITEM']
        for i,answer in enumerate(student['TX_RESPOSTAS_CH']):
            fill_data_for_question(items_prova[i],student,answer)
            
    if not student['TX_RESPOSTAS_CN']==np.nan:
        items_prova = answer_mapping.loc[(student['CO_PROVA_CN'],lingua),'CO_ITEM']
        for i,answer in enumerate(student['TX_RESPOSTAS_CN']):
            fill_data_for_question(items_prova[i],student,answer)
            
    if not student['TX_RESPOSTAS_LC']==np.nan:
        items_prova = answer_mapping.loc[(student['CO_PROVA_LC'],lingua),'CO_ITEM']
        for i,answer in enumerate(student['TX_RESPOSTAS_LC']):
            fill_data_for_question(items_prova[i],student,answer)
            
    if not student['TX_RESPOSTAS_MT']==np.nan:
        items_prova = answer_mapping.loc[(student['CO_PROVA_MT'],lingua),'CO_ITEM']
        for i,answer in enumerate(student['TX_RESPOSTAS_MT']):
            fill_data_for_question(items_prova[i],student,answer)

100%|██████████████████████████████████████████████████████████████████████| 1896156/1896156 [11:51<00:00, 2665.50it/s]


In [13]:
with open('MicrodadosAlunos/answer_data.csv','w') as file:
    file.write('CO_ITEM,ID,RESPOSTA\n')
    for i,line in enumerate(tqdm(student_answers)):
        file.write(f"{line['CO_ITEM']},{line['ID']},{line['RESPOSTA']}\n")

100%|████████████████████████████████████████████████████████████████| 341308080/341308080 [09:23<00:00, 605870.10it/s]


In [14]:
with open('MicrodadosAlunos/student_data.csv','w') as file:
    file.write('ID,NU_INSCRICAO,SG_UF_PROVA,TP_ESCOLA,RENDA,TP_SEXO,TP_COR_RACA\n')
    for i,line in enumerate(tqdm(students)):
        file.write(f"{line['ID']},{line['NU_INSCRICAO']},{line['SG_UF_PROVA']},{line['TP_ESCOLA']},{line['RENDA']},{line['TP_SEXO']},{line['TP_COR_RACA']}\n")

100%|████████████████████████████████████████████████████████████████████| 1896156/1896156 [00:13<00:00, 142809.58it/s]
